In [24]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from bs4 import BeautifulSoup
import requests
import json
from IPython.display import Markdown, display, update_display

In [15]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("Good")
else:
    print("Bad")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

Good


In [3]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(['script', 'style', 'img', 'input']):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ''
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

In [5]:
link_system_prompt = "웹페이지에서 찾은 링크 목록이 제공됩니다. \
회사 브로셔에 포함할 가장 관련성 있는 링크를 결정할 수 있습니다. \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "이 예와 같이 JSON으로 응답해야 합니다.:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

웹페이지에서 찾은 링크 목록이 제공됩니다. 회사 브로셔에 포함할 가장 관련성 있는 링크를 결정할 수 있습니다. such as links to an About page, or a Company page, or Careers/Jobs pages.
이 예와 같이 JSON으로 응답해야 합니다.:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"다음은 웹사이트의 링크 목록입니다. {website.url} - "
    user_prompt += "회사 소개 브로셔에 대한 관련 웹 링크가 어느 것인지 결정하여 JSON 형식의 전체 https URL을 응답해 주세요. \
서비스 약관, 개인정보 보호정책, 이메일 링크는 포함하지 마세요.\n"
    user_prompt += "링크(일부는 상대 링크일 수 있음):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

다음은 웹사이트의 링크 목록입니다. https://edwarddonner.com - 회사 소개 브로셔에 대한 관련 웹 링크가 어느 것인지 결정하여 JSON 형식의 전체 https URL을 응답해 주세요. 서비스 약관, 개인정보 보호정책, 이메일 링크는 포함하지 마세요.
링크(일부는 상대 링크일 수 있음):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://ed

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [13]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/smolagents',
 '/microsoft/phi-4',
 '/deepseek-ai/DeepSeek-V3',
 '/hexgrad/Kokoro-82M',
 '/black-forest-labs/FLUX.1-dev',
 '/PowerInfer/SmallThinker-3B-Preview',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/osanseviero/gemini-coder',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/akhaliq/anychat',
 '/spaces/reach-vb/2024-ai-timeline',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/cfahlgren1/react-code-instructions',
 '/datasets/agibot-world/AgiBotWorld-Alpha',
 '/datasets/PowerInfer/QWQ-LONGCOT-500K',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '

In [17]:
get_links('https://huggingface.co')

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [19]:
def get_all_details(url):
    result = 'Landing page:\n'
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links['links']:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
Use Ollama with GGUF Models from the HF Hub
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
microsoft/phi-4
Updated
1 day ago
•
23.3k
•
845
deepseek-ai/DeepSeek-V3
Updated
11 days ago
•
81.5k
•
1.61k
hexgrad/Kokoro-82M
Updated
4 days ago
•
3.41k
•
424
black-forest-labs/FLUX.1-dev
Updated
Aug 16, 2024
•
1.1M
•
7.87k
PowerInfer/SmallThinker-3B-Preview
Updated
4 days ago
•
10.4k
•
322
Browse 400k+ models
Spaces
Running
on
Zero
2.78k


In [21]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nUse Ollama with GGUF Models from the HF Hub\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmicrosoft/phi-4\nUpdated\n1 day ago\n•\n23.3k\n•\n846\ndeepseek-ai/DeepSeek-V3\nUpdated\n11 days ago\n•\n81.5k\n•\n1.61k\nhexgrad/Kokoro-82M\nUpdated\n4 days ago\n•\n3.41k\n•\n424\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16, 2024\n•\n1.1M\n•\n7.87k\nPowerInfer/SmallThinker-3B-Preview\nUpdat

In [25]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [26]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'contact page', 'url': 'https://huggingface.com/contact'}]}


# Welcome to the Hugging Face Universe! 🌌

### Where AI Dreams Come True! 

At **Hugging Face**, we believe in a world where AI is the epitome of cuddly innovation. Forget robots taking over the world – we’re all about collaborative hugging, not body slamming! Whether you're a customer, investor, or future team member, here’s what you need to know about our AI paradise.

---

## 🤖 Our Community Is Giggle-Worthy!
At Hugging Face, we’re not just a bunch of brainy folks coding in dark basements. We’re a **community**! We openly share **models**, **datasets**, and joke-sparking **Spaces** (which is not where you keep your avocados). Join over **50,000** organizations, including giants like Microsoft, Google, and Slack. 

## 🏆 The Great Model & Dataset Hoedown!
Ever tried going through **400k+ models**? It's like searching for a needle in a haystack but with more math and less hay. Pop in, browse, and you might just find **microsoft/phi-4** or our newest favorite, **PowerInfer/SmallThinker-3B-Preview**. 

And don't forget **150k datasets** – perfect for when you need to feed your models or just to distract your friends with random facts about AI!

## 🎨 Spaces for Creativity
Working from home? Dive into our **Spaces** feature! Create jaw-dropping 3D images, plug into coding wizards, or just showcase your new cat GIF generator. Who knew AI could be so much fun? Don’t worry, they won’t judge you even if you name your project “Grumpy Cat 3D”.

## 🚀 Career Opportunities – Join the Hugging Hugger Squad!
Thinking of joining our ranks? We offer a vibrant **company culture** where creativity and cuddles collide! Whether you're into data wrangling, model training, or just want to be part of the world's biggest hug (just kidding, we won't crowd you), there's a place for you here.

**Current Openings include:**
- AI Unicorn Tamer 🦄
- Data Drama Queen/King 👑
- Model Whisperer 🎤

Work with a team that connects on a personal level while tackling the world’s deepest AI mysteries. Plus, complimentary hugs – virtual and real!

## 🎉 Why Choose Hugging Face?
- Open Source: Because sharing is caring!
- Fun, diverse team: We speak 300 dialects of 'data' and 22 languages of humor.
- Futuristic Culture: Work on the frontier of AI today to build the utopia of tomorrow!

So, whether you're a sector-savvy investor ready to cash in on the AI gold rush or an aspiring genius looking to collaborate with the finest minds, **Welcome to Hugging Face**! Let’s build the future – one hug at a time! 🤗🌈

---

### 🌐 Get in Touch! 
- **Website:** [huggingface.co](https://huggingface.co)
- **Follow us on:** Twitter, GitHub, and Discord
- **Apply Now:** Join for the hugs, stay for the models!

**P.S.** Our hug quota is limitless – come join us today! 🎉

In [27]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ''
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face: The AI Community Building the Future! 🤗

### Are you ready to unlock the magic of AI? Because we've got *hugs* for everyone!

---
### Who Are We?
At Hugging Face, we believe in the potential of AI to shape the future - one model (and hug) at a time! We're not just a tech company; we're a vibrant community where researchers, developers, and AI enthusiasts unite to graft brains and algorithms. 

**Fun Fact:** We have over **400,000 models** on our platform—try picking favorites. It's like being at a family reunion with lost cousins you never wanted to meet!

---

### What Do We Offer? 
- **Models Galore:** From Microsoft’s latest super-duper model to the mysterious black forest-labs’ FLUX.1-dev, you're in for a treat!
  
- **Datasets to Drool Over:** With **100,000+ datasets** at your fingertips, you’ll never run out of data to dig into. It’s like a buffet for machine learning!

- **Spaces:** Create and collaborate in our cozy 🌌 virtual spaces where every AI journey begins... or takes a long coffee break.

---

### Our Customers Are the Best! 🏆
Join over **50,000 organizations** including big names like Amazon, Google, Microsoft, and… well, we keep all the cool kids in our filers! Whether you’re a small startup or a bona fide tech giant, we’ve got something for everyone with enterprise-grade solutions that make your AI dreams come true.

---

### Come Work With Us! 🎉
We’re hiring! Join a team that values innovation, creativity, and the occasional meme about math being hard. Plus, you’ll find colleagues who share a love for neural networks—and actual hugs. 

- **Culture:** Flexible work hours, coffee that rivals your local barista, and a casual dress code (yes, pajamas are accepted during virtual meetings). 
- **Perks:** We promise to keep the fridge stocked with snacks and the breakroom filled with good vibes!

---

### Why Choose Hugging Face?
At the end of the day, even when you feel like you're in a noisy machine-learning jungle, Hugging Face is your trusty sidekick guiding you on your journey—the Robin to your Batman. 

So what are you waiting for? Join us as we embrace the future of AI—one hug at a time!

---
**Want more information?**  
Visit [Hugging Face](https://huggingface.co) or give us a friendly wave on social media! 🌟 

